# Lecture 2: Introduction to Feature Engineering

## Setup

In [ ]:
import numpy as np 
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [ ]:
articles = pd.read_csv('articles_sample5.csv.gz')
customers = pd.read_csv('customers_sample5.csv.gz')
# sample_submisison = pd.read_csv('../data/sample_submission.csv')
transactions = pd.read_csv('transactions_sample5.csv.gz')

## The H&M Dataset

In [ ]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77001 entries, 0 to 77000
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   article_id                    77001 non-null  int64 
 1   product_code                  77001 non-null  int64 
 2   prod_name                     77001 non-null  object
 3   product_type_no               77001 non-null  int64 
 4   product_type_name             77001 non-null  object
 5   product_group_name            77001 non-null  object
 6   graphical_appearance_no       77001 non-null  int64 
 7   graphical_appearance_name     77001 non-null  object
 8   colour_group_code             77001 non-null  int64 
 9   colour_group_name             77001 non-null  object
 10  perceived_colour_value_id     77001 non-null  int64 
 11  perceived_colour_value_name   77001 non-null  object
 12  perceived_colour_master_id    77001 non-null  int64 
 13  perceived_colour

In [ ]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68599 entries, 0 to 68598
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             68599 non-null  object 
 1   FN                      23810 non-null  float64
 2   Active                  23205 non-null  float64
 3   club_member_status      68272 non-null  object 
 4   fashion_news_frequency  67797 non-null  object 
 5   age                     67769 non-null  float64
 6   postal_code             68599 non-null  object 
dtypes: float64(3), object(4)
memory usage: 3.7+ MB


In [ ]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1585334 entries, 0 to 1585333
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   t_dat             1585334 non-null  object 
 1   customer_id       1585334 non-null  object 
 2   article_id        1585334 non-null  int64  
 3   price             1585334 non-null  float64
 4   sales_channel_id  1585334 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 60.5+ MB


In [ ]:
# X = transactions.merge(customers, how='inner', on='customer_id')
# X = X.merge(articles, how='inner', on='article_id')

### Creating Samples 
If you would rather work with samples instead of the whole dataset (while prototyping your code). You can use the code below:

In [ ]:
# Adapted from: https://www.kaggle.com/code/paweljankiewicz/hm-create-dataset-samples
# This extracts three sampled datasets, containing 0.1%, 1% and 5% of all users and their transactions, and the associated articles.
# for sample_repr, sample in [("01", 0.001), ("1", 0.01), ("5", 0.05)]:
#     print(sample)
#     customers_sample = customers.sample(int(customers.shape[0]*sample), replace=False)
#     customers_sample_ids = set(customers_sample["customer_id"])
#     transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
#     articles_sample_ids = set(transactions_sample["article_id"])
#     articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]
#     customers_sample.to_csv(f"../data/customers_sample{sample_repr}.csv.gz", index=False)
#     transactions_sample.to_csv(f"../data/transactions_sample{sample_repr}.csv.gz", index=False)
#     articles_sample.to_csv(f"../data/articles_sample{sample_repr}.csv.gz", index=False)

In [ ]:
# articles_sample = pd.read_csv('../data/articles_sample01.csv.gz')
# customers_sample = pd.read_csv('../data/customers_sample01.csv.gz')
# transactions_sample = pd.read_csv('../data/transactions_sample01.csv.gz')

In [ ]:
# customers_sample.info()

In [ ]:
# transactions_sample.info()

## A Simplified Task: Binary Classification

The task of predicting which 12 items users are most likely to buy in the next week is difficult to translate to a traditional classification machine learning setting. 
To obtain the 12 items a user is most likely to buy, we need to make predictions for all items (or the ones selected by a baseline) and select the 12 that have the highest predicted scores.

In this assignment, we'll consider a simplified task: Predict whether a user ordered a single item or not, based on the features of the user and the item. 
We provide a baseline logistic regression model below, but haven't done much feature preprocessing or engineering!
Initially, it is always best to focus your efforts on getting your features in the right shape and setting up the right validation scheme and baselines.
Once you are sure that your features add value and your validation scheme is correct, then you typically move on to trying more elaborate models.

### Creating the Dataset

In [ ]:
# If you'd rather use a sample. Uncomment the following code:
# transactions = transactions_sample
# customers = customers_sample
# articles = articles_sample

In [ ]:
transactions['ordered'] = 1

The problem setting is an example of a "PU learning" problem, i.e. only positives are labeled, everything else is unlabeled (and can be either positive or negative). 
Of course, we cannot train a classifier with just positive samples: The classifier will just learn that everything is positive.
Therefore, we need to manually generate negative samples.

Below, we use a simple random negative sampling strategy.
We want to create a balanced dataset, meaning that we have just as many positives as negatives.
This makes sure that the classifier will not benefit from predicting the positive/negative class more often than the other.
Realistically, the amount of positive samples is of course many times smaller than the amount of unlabeled, possibly negative instances.


If you want to try your hand at a more complex negative sampling strategy, you may want to check out this blog as a starting point: https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7.



In [ ]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,0021da829b898f82269fc51feded4eac2129058ee95bd7...,649356002,0.027441,1,1
1,2018-09-20,0021da829b898f82269fc51feded4eac2129058ee95bd7...,579941002,0.019814,1,1
2,2018-09-20,0021da829b898f82269fc51feded4eac2129058ee95bd7...,629760002,0.015237,1,1
3,2018-09-20,0021da829b898f82269fc51feded4eac2129058ee95bd7...,625229004,0.019814,1,1
4,2018-09-20,004888668b9458668aa3234d17337671673599f7f84d36...,662857005,0.042356,1,1


In [ ]:
# What's happening here? 
# We're creating negative samples. I.e. we're creating transactions that didn't actually occur.
# First, we need to know which interactions did occur:
positive_pairs = list(map(tuple, transactions[['customer_id', 'article_id']].drop_duplicates().values))

In [ ]:
# Then we need to know what every synthetic transaction should contain: a date, a customer_id, an article_id, price, sales_channel_id. We will set ordered = 0, as these transactions didn't really occur.
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,0021da829b898f82269fc51feded4eac2129058ee95bd7...,649356002,0.027441,1,1
1,2018-09-20,0021da829b898f82269fc51feded4eac2129058ee95bd7...,579941002,0.019814,1,1
2,2018-09-20,0021da829b898f82269fc51feded4eac2129058ee95bd7...,629760002,0.015237,1,1
3,2018-09-20,0021da829b898f82269fc51feded4eac2129058ee95bd7...,625229004,0.019814,1,1
4,2018-09-20,004888668b9458668aa3234d17337671673599f7f84d36...,662857005,0.042356,1,1


In [ ]:
# Extract real values
real_dates = transactions["t_dat"].unique()
real_customers = transactions["customer_id"].unique()
real_articles = transactions["article_id"].unique()
real_channels = transactions["sales_channel_id"].unique()
article_and_price = transactions[["article_id","price"]].drop_duplicates("article_id").set_index("article_id").squeeze()

In [ ]:
# How many negatives do we need to sample?
num_neg_pos = transactions.shape[0]
print(num_neg_pos)

1585334


In [ ]:
# Sampling negatives by selecting random users, articles, dates and sales channel:
# Note: This is quite naive. Some articles may not even have been available at the date we are sampling.
random.seed(42)

# Afterwards, we need to remove potential duplicates, so we'll sample too many.
num_neg_samples = int(num_neg_pos * 1.1)

# Sample each of the independent attributes.
neg_dates = np.random.choice(real_dates, size=num_neg_samples)
neg_articles = np.random.choice(real_articles, size=num_neg_samples)
neg_customers = np.random.choice(real_customers, size=num_neg_samples)
neg_channels = np.random.choice(real_channels, size=num_neg_samples)
ordered = np.array([0] * num_neg_samples)
# Assign to every article a real price.
neg_prices = article_and_price[neg_articles].values

In [ ]:
neg_transactions = pd.DataFrame([neg_dates, neg_customers, neg_articles, neg_prices, neg_channels, ordered], index=transactions.columns).T

In [ ]:
# Result:
neg_transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2019-06-10,8ec48266c60fb104df24c8e9637760de9ad1416de4d304...,730048001,0.022017,1,0
1,2019-08-19,b87a66430e43d2faa27ea1a4ea8b7086d47e7ff4b5af01...,576396001,0.022017,1,0
2,2019-01-07,f70d96cb02b6a4ef871dd6f209b6020cd5c4a67e3586fe...,819022001,0.025407,2,0
3,2020-04-26,35713ac634ac2913946f35a3b250773c6f2cab337d6ad3...,922753001,0.016932,1,0
4,2019-12-13,989bd07a42a143ed34194a789e8f7ce8b1d13b28afca02...,827704004,0.027102,1,0


In [ ]:
neg_transactions.shape

(1743867, 6)

In [ ]:
# Remove random negative samples that actually coincide with positives
df = neg_transactions[
    ~neg_transactions.set_index(["customer_id", "article_id"]).index.isin(positive_pairs)
]

# Remove any excess
chosen_neg_transactions = df.sample(num_neg_pos)

In [ ]:
# Concat the negative samples to the positive samples:
transactions = pd.concat([transactions, chosen_neg_transactions])
transactions = transactions.merge(customers, how="inner", on='customer_id')
transactions = transactions.merge(articles, how="inner", on='article_id')

In [ ]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3170668 entries, 0 to 3170667
Data columns (total 36 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   t_dat                         object 
 1   customer_id                   object 
 2   article_id                    object 
 3   price                         object 
 4   sales_channel_id              object 
 5   ordered                       object 
 6   FN                            float64
 7   Active                        float64
 8   club_member_status            object 
 9   fashion_news_frequency        object 
 10  age                           float64
 11  postal_code                   object 
 12  product_code                  int64  
 13  prod_name                     object 
 14  product_type_no               int64  
 15  product_type_name             object 
 16  product_group_name            object 
 17  graphical_appearance_no       int64  
 18  graphical_appearance_n

### Basic Preprocessing
Some very basic preprocessing.

In [ ]:
# I'm dropping a lot of columns, use them in your engineering tasks!
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered']].copy()
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,0021da829b898f82269fc51feded4eac2129058ee95bd7...,41.0,649356002,1,0.027441,1
1,30715fdcf3a118b192dadc87c7e486600fe75c5220cec4...,25.0,649356002,1,0.027441,0
2,6760968bb5134b4fab2c466b659aeaa05a25658adfe872...,27.0,649356002,2,0.027441,0
3,bd98d959ff637fbdef53211ab414ff53343dfbbba5605c...,47.0,649356002,1,0.030492,1
4,1b35dae03781a752ce6b250219d2513ba19e7d0b04586c...,43.0,649356002,1,0.02439,1


In [ ]:
# Does it make sense to label encode?
# Label encoding the customer and article IDs:
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()

In [ ]:
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])

In [ ]:
# If you want to go back to the original encoding:
customer_encoder.inverse_transform([2])

array(['0002b7a7ab270a638fcb2eb5899c58696db24d9d954ddb43683dd6b0ffa292f0'],
      dtype=object)

In [ ]:
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,36,41.0,23268,1,0.027441,1
1,12830,25.0,23268,1,0.027441,0
2,27495,27.0,23268,2,0.027441,0
3,50300,47.0,23268,1,0.030492,1
4,7105,43.0,23268,1,0.02439,1


In [ ]:
# Can you come up with a NaN strategy that makes sense for each column in the dataset?
# Imputing all NaN values with zeros:
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()

False

In [ ]:
# Does it make sense to one-hot encode?
# One-hot-encoding sales_channel_id:
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id'])

In [ ]:
transactions_processed.head()

,customer_id,age,article_id,price,ordered,sales_channel_id_1,sales_channel_id_2
0,36,41.0,23268,0.027441,1,1,0
1,12830,25.0,23268,0.027441,0,1,0
2,27495,27.0,23268,0.027441,0,0,1
3,50300,47.0,23268,0.030492,1,1,0
4,7105,43.0,23268,0.024390,1,1,0


In [ ]:
# Creating a Train / Test Split:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)

In [ ]:
X_train.head()

,customer_id,age,article_id,price,sales_channel_id_1,sales_channel_id_2
750811,54275,23.0,64435,0.016932,1,0
247514,53170,23.0,2497,0.033881,1,0
555616,22270,25.0,28580,0.025407,0,1
2432041,47180,33.0,45435,0.008458,0,1
2245626,40898,45.0,25849,0.011847,0,1


In [ ]:
y_train.head()

750811     1
247514     1
555616     1
2432041    1
2245626    0
Name: ordered, dtype: int64

## Baseline Model

In [ ]:
# Will take a few minutes to run, if you're using the whole dataset:
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)

In [ ]:
baseline.predict_proba(X_test)

array([[0.51111775, 0.48888225],
       [0.50408743, 0.49591257],
       [0.51062711, 0.48937289],
       ...,
       [0.5054505 , 0.4945495 ],
       [0.49855868, 0.50144132],
       [0.4809243 , 0.5190757 ]])

In [ ]:
y_test

1144932    1
573054     1
422719     0
669009     1
3059264    0
          ..
2896010    1
2804002    0
2285802    0
1192463    1
2048768    0
Name: ordered, Length: 317067, dtype: int64

In [ ]:
# Mean Accuracy:
baseline.score(X_test, y_test)
# As you can seen, the accuracy is ~0.51. In other words, the classifier predicts correctly 51% of the time whether a customer did or din't buy an item.
# Can you improve this baseline logistic regression model by doing better preprocessing and generating new features?
# Also think about my steps! Did it make sense to include the article and customer ids? (And things like that)

0.5092519877502232

In [ ]:
# Classification Metrics:
predictions = baseline.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.51      0.45      0.48    158593
           1       0.51      0.57      0.54    158474

    accuracy                           0.51    317067
   macro avg       0.51      0.51      0.51    317067
weighted avg       0.51      0.51      0.51    317067



In [ ]:
predictions

array([0, 0, 0, ..., 0, 1, 1])

## Assignment: Feature engineering
**TODO:** 
- In groups (of 2-3 students), think about (a few) features that can be engineered (preprocess and generate new features). Divide the work!
- Do these engineered features improve the baseline model?
- Add your thoughts & results to a slide deck for discussion next week (again, 1 slide per person).


Make a dataframe to learn the popularity of each bought item.

In [ ]:
popularity = transactions.loc[transactions["ordered"] == 1][["article_id"]]
popularity = popularity["article_id"].value_counts().rename_axis("article_id").reset_index(name="count")

A list with key words in the description. I use this to get a feature with material type.

In [ ]:
materials = ["nylon", "cotton", "Microfibre", "jeans", "silicone", "synthetic", "wool", "plastic", "silk", "leather", "lace"]

Make a dataframe with all the features. Add the popularity.

In [ ]:
transactions_processed = transactions[['customer_id', 'article_id', 'sales_channel_id', 'price', 'ordered', "age", "detail_desc", "graphical_appearance_no", "colour_group_code", "t_dat"]].copy()
transactions_processed = transactions_processed.merge(popularity, how="inner", on='article_id')


Get the month the item is bought and the popularity in that month.

In [ ]:
transactions_processed["t_dat"] = pd.to_datetime(transactions_processed["t_dat"])
transactions_processed["t_dat"] = transactions_processed["t_dat"].dt.month

In [ ]:
for i in range(1,13):
  month_date = transactions_processed.loc[transactions_processed["t_dat"] == (i)][["article_id", "ordered"]]
  month_pop = month_date.loc[month_date["ordered"] == 1][["article_id"]]
  month_pop = month_pop["article_id"].value_counts().rename_axis("article_id").reset_index(name=f"month{i}_pop")
  transactions_processed = transactions_processed.merge(month_pop, how="inner", on='article_id')

Bin the age and price.

In [ ]:
transactions_processed["age_bin"] = pd.cut(transactions_processed['age'], [0, 21, 35,45, 55, 65, 200], labels=['21', '35', '45', '55','65', '200'], right=False)
transactions_processed["price_bin"] = pd.cut(transactions_processed['price'], [0, 0.02, 0.04, 0.06,0.08,1], labels=[0,  0.02,  0.04,0.06,0.08], right=False)

Make a column for each material type. 

In [ ]:
count = 0
for material in materials:
  transactions_processed[material] = [1 if material in str(a) else 0 for a in transactions_processed["detail_desc"]]



Drop procesed columns.

In [ ]:
transactions_processed = transactions_processed.drop("detail_desc", axis=1)
transactions_processed = transactions_processed.drop("age", axis=1)
transactions_processed = transactions_processed.drop("price", axis=1)
transactions_processed = transactions_processed.drop("t_dat", axis=1)

transactions_processed.head()

,customer_id,article_id,sales_channel_id,ordered,graphical_appearance_no,colour_group_code,count,month1_pop,month2_pop,month3_pop,...,cotton,Microfibre,jeans,silicone,synthetic,wool,plastic,silk,leather,lace
0,004888668b9458668aa3234d17337671673599f7f84d36...,372860001,2,1,1010016,9,1511,124,132,142,...,1,0,0,0,0,0,0,0,0,0
1,1297a1eba28727ea40a7c652d54edb570e5560742c143b...,372860001,1,1,1010016,9,1511,124,132,142,...,1,0,0,0,0,0,0,0,0,0
2,16501613daf68dad9dc9bd34880b12efe4f473196b9f33...,372860001,1,1,1010016,9,1511,124,132,142,...,1,0,0,0,0,0,0,0,0,0
3,1aa1dfb041052ea08cd666e5509de21bf26efa6406bfd2...,372860001,1,1,1010016,9,1511,124,132,142,...,1,0,0,0,0,0,0,0,0,0
4,1aa1dfb041052ea08cd666e5509de21bf26efa6406bfd2...,372860001,1,1,1010016,9,1511,124,132,142,...,1,0,0,0,0,0,0,0,0,0


Encode columns in labels.

In [ ]:
from pandas.core import aggregation
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()
age_encoder = preprocessing.LabelEncoder()
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])
transactions_processed['age_bin'] = age_encoder.fit_transform(transactions_processed['age_bin'])


Preprocess.

In [ ]:
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id', 'graphical_appearance_no', 'price_bin', "colour_group_code"])

Test the features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)
engineered = LogisticRegression(random_state=42)
engineered = engineered.fit(X_train, y_train)
engineered.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.9005122486671081